<a href="https://colab.research.google.com/github/ArmourG2/Colab-TGW/blob/main/Edit_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM text generation notebook for Google Colab

Version: GPU

This notebook uses **oobabooga**'s [text-generation-webui project](https://github.com/oobabooga/text-generation-webui).

Run all the cells and a public URL will appear at the bottom.

## Useful links

* [Web UI wiki](https://github.com/oobabooga/text-generation-webui/wiki)
* [Hugging Face model hub](https://huggingface.co/models)
* [Chatbot character creator](https://oobabooga.github.io/character-creator.html)

## Credits

* [oobabooga's notebook](https://github.com/oobabooga/AI-Notebooks)
* [text-generation-webui](https://github.com/oobabooga/text-generation-webui)

In [ ]:
#@title 1. Keep the tab alive { display-mode: "form" }

#@markdown This can prevent Colab from terminating your runtime. Press play on the music player that will appear below:

%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
#@title 2. Install the web UI

import os
import shutil
from google.colab import drive

Save_data_to_Google_Drive = True #@param {type:"boolean"}
Save_To_Google = True #@param {type:"boolean"} 
Install_GPTQ = True #@param {type:"boolean"}
Reinstall_all = True #@param {type:"boolean"}
Reinstall = False #@param {type:"boolean"}

if Save_data_to_Google_Drive:
  import os
  import shutil
  from google.colab import drive
  drive.mount("/content/drive")
  drive_folder = "/content/drive/MyDrive"

if Save_To_Google:
  import os
  import shutil
  from google.colab import drive
  drive.mount("/content/drive")
  drive_folder = "/content/drive/MyDrive"
  repo_path = "/content/drive/MyDrive/Colab-TextGen"  # Update the Google Drive folder path
  model_dir = '/content/drive/MyDrive/Colab-TextGen/models'
  gptq_dir = '/content/drive/MyDrive/Colab-TextGen/GPTQ-for-LLaMa'
  if not os.path.exists(repo_path):
    os.mkdir(repo_path)
  else:
    %cd $repo_path
    ![[ ! -d .git ]] && git clone https://github.com/oobabooga/text-generation-webui .
    !git reset --hard
    !git pull
    !pip install --no-cache-dir --progress-bar=off -r requirements.txt
    !pip install --no-cache-dir --progress-bar=off -r extensions/api/requirements.txt
    !wget https://oobabooga.github.io/settings-colab.json -O settings-colab.json
else:
  import os
  repo_path = "/content/Colab-TextGen"
  model_dir = '/content/Colab-TextGen/models'
  gptq_dir = '/content/Colab-TextGen/GPTQ-for-LLaMa'

if Reinstall_all:
  !rm -r $repo_path

if not os.path.exists(repo_path):
  os.mkdir(repo_path)

%cd $repo_path
![[ ! -d .git ]] && git clone https://github.com/oobabooga/text-generation-webui .
!git reset --hard
!git pull
!pip install --no-cache-dir --progress-bar=off -r requirements.txt
!pip install --no-cache-dir --progress-bar=off -r extensions/api/requirements.txt
!wget https://oobabooga.github.io/settings-colab.json -O settings-colab.json

# Install requirements
!pip install -r requirements.txt
!pip install -r extensions/google_translate/requirements.txt
!pip install -r extensions/silero_tts/requirements.txt
!pip install flask-cloudflared
print(f"\033[1;32;1m\n --> If you see a warning about \"pydevd_plugins\", just ignore it and move on to Step 3. There is no need to restart the runtime.\n\033[0;37;0m")

if Save_data_to_Google_Drive:
  if not os.path.exists(f"{drive_folder}/oobabooga-data"):
    os.mkdir(f"{drive_folder}/oobabooga-data")
  if not os.path.exists(f"{drive_folder}/oobabooga-data/logs"):
    os.mkdir(f"{drive_folder}/oobabooga-data/logs")
  if not os.path.exists(f"{drive_folder}/oobabooga-data/softprompts"):
    os.mkdir(f"{drive_folder}/oobabooga-data/softprompts")
  if not os.path.exists(f"{drive_folder}/oobabooga-data/characters"):
    shutil.move(f"{repo_path}/characters", f"{drive_folder}/oobabooga-data/characters")
  else:
    !rm -r "$repo_path/characters"
    
  !rm -r "$repo_path/softprompts"
  !ln -sf "$drive_folder/oobabooga-data/logs" "$repo_path/logs"
  !ln -sf "$drive_folder/oobabooga-data/softprompts" "$repo_path/softprompts"
  !ln -sf "$drive_folder/oobabooga-data/characters" "$repo_path/characters"

if Install_GPTQ:
  if Save_To_Google:
    if os.path.exists(gptq_dir):
            %cd {gptq_dir}
            !git clone https://github.com/oobabooga/GPTQ-for-LLaMa.git -b cuda
            %cd GPTQ-for-LLaMa
            !pip install ninja
            !pip install -r requirements.txt
            !python setup_cuda.py install
    else:
            %cd gptq_dir
            !mkdir repositories
            !git clone https://github.com/qwopqwop200/GPTQ-for-LLaMa
            !ln -s GPTQ-for-LLaMa gptq_dir
            %cd GPTQ-for-LLaMa
            !pip install ninja
            !pip install -r requirements.txt
            !python setup_cuda.py install
  else:
    %cd repo_path
    %mkdir repositories
    !git clone https://github.com/qwopqwop200/GPTQ-for-LLaMa
    !mkdir -p text-generation-webui/repositories
    !ln -s GPTQ-for-LLaMa text-generation-webui/repositories/GPTQ-for-LLaMa
    %cd GPTQ-for-LLaMa
    !pip install ninja
    !pip install -r requirements.txt
    !python setup_cuda.py install

In [ ]:
#@title 3. Select a model

model_download = "" #@param {type:"string"}
Redownload = False #@param {type: "boolean"}
Delete_all_models = False #@param {type: "boolean"}

#Download model
%cd {repo_path}
!python download-model.py {model_download}

if Save_data_to_Google_Drive or save_everything_to_google_drive:
    drive_NOT_mounted = False
else:
    drive_NOT_mounted = True

if Delete_all_models:
  import os
  if os.path.isdir(model_dir):
    %cd $model_dir
    !find . -mindepth 1 -maxdepth 1 -type d -exec rm -rv "{}" \;
  else:
    print("Not a valid models directory.")

if drive_NOT_mounted:
  import os
print("Available Models")
print(os.listdir(model_dir))

In [ ]:
#@title 3. Launch

import json

#@markdown if you dont know what to enter the previous cell should have printed available inputs <br> paste it here
model_load = "4bit_WizardLM-13B-Uncensored-4bit-128g" #@param {type:"string"}
# Parameters
#auto_devices = False #@param {type:"boolean"}
load_4bit_models = True #@param {type:"boolean"}

groupsize_128 = True #@param {type:"boolean"}
load_in_8bit = False #@param {type:"boolean"}
chat = False #@param {type:"boolean"}
api = False #@param {type:"boolean"}
public_api = True #@param {type:"boolean"}

text_streaming = False #@param {type:"boolean"}
activate_silero_text_to_speech = False #@param {type:"boolean"}
activate_sending_pictures = False #@param {type:"boolean"}
activate_character_bias = False #@param {type:"boolean"}
chat_language = "English" # @param ['Afrikaans', 'Albanian', 'Amharic', 'Arabic', 'Armenian', 'Azerbaijani', 'Basque', 'Belarusian', 'Bengali', 'Bosnian', 'Bulgarian', 'Catalan', 'Cebuano', 'Chinese (Simplified)', 'Chinese (Traditional)', 'Corsican', 'Croatian', 'Czech', 'Danish', 'Dutch', 'English', 'Esperanto', 'Estonian', 'Finnish', 'French', 'Frisian', 'Galician', 'Georgian', 'German', 'Greek', 'Gujarati', 'Haitian Creole', 'Hausa', 'Hawaiian', 'Hebrew', 'Hindi', 'Hmong', 'Hungarian', 'Icelandic', 'Igbo', 'Indonesian', 'Irish', 'Italian', 'Japanese', 'Javanese', 'Kannada', 'Kazakh', 'Khmer', 'Korean', 'Kurdish', 'Kyrgyz', 'Lao', 'Latin', 'Latvian', 'Lithuanian', 'Luxembourgish', 'Macedonian', 'Malagasy', 'Malay', 'Malayalam', 'Maltese', 'Maori', 'Marathi', 'Mongolian', 'Myanmar (Burmese)', 'Nepali', 'Norwegian', 'Nyanja (Chichewa)', 'Pashto', 'Persian', 'Polish', 'Portuguese (Portugal, Brazil)', 'Punjabi', 'Romanian', 'Russian', 'Samoan', 'Scots Gaelic', 'Serbian', 'Sesotho', 'Shona', 'Sindhi', 'Sinhala (Sinhalese)', 'Slovak', 'Slovenian', 'Somali', 'Spanish', 'Sundanese', 'Swahili', 'Swedish', 'Tagalog (Filipino)', 'Tajik', 'Tamil', 'Telugu', 'Thai', 'Turkish', 'Ukrainian', 'Urdu', 'Uzbek', 'Vietnamese', 'Welsh', 'Xhosa', 'Yiddish', 'Yoruba', 'Zulu']


activate_google_translate = (chat_language != "English")

language_codes = {'Afrikaans': 'af', 'Albanian': 'sq', 'Amharic': 'am', 'Arabic': 'ar', 'Armenian': 'hy', 'Azerbaijani': 'az', 'Basque': 'eu', 'Belarusian': 'be', 'Bengali': 'bn', 'Bosnian': 'bs', 'Bulgarian': 'bg', 'Catalan': 'ca', 'Cebuano': 'ceb', 'Chinese (Simplified)': 'zh-CN', 'Chinese (Traditional)': 'zh-TW', 'Corsican': 'co', 'Croatian': 'hr', 'Czech': 'cs', 'Danish': 'da', 'Dutch': 'nl', 'English': 'en', 'Esperanto': 'eo', 'Estonian': 'et', 'Finnish': 'fi', 'French': 'fr', 'Frisian': 'fy', 'Galician': 'gl', 'Georgian': 'ka', 'German': 'de', 'Greek': 'el', 'Gujarati': 'gu', 'Haitian Creole': 'ht', 'Hausa': 'ha', 'Hawaiian': 'haw', 'Hebrew': 'iw', 'Hindi': 'hi', 'Hmong': 'hmn', 'Hungarian': 'hu', 'Icelandic': 'is', 'Igbo': 'ig', 'Indonesian': 'id', 'Irish': 'ga', 'Italian': 'it', 'Japanese': 'ja', 'Javanese': 'jw', 'Kannada': 'kn', 'Kazakh': 'kk', 'Khmer': 'km', 'Korean': 'ko', 'Kurdish': 'ku', 'Kyrgyz': 'ky', 'Lao': 'lo', 'Latin': 'la', 'Latvian': 'lv', 'Lithuanian': 'lt', 'Luxembourgish': 'lb', 'Macedonian': 'mk', 'Malagasy': 'mg', 'Malay': 'ms', 'Malayalam': 'ml', 'Maltese': 'mt', 'Maori': 'mi', 'Marathi': 'mr', 'Mongolian': 'mn', 'Myanmar (Burmese)': 'my', 'Nepali': 'ne', 'Norwegian': 'no', 'Nyanja (Chichewa)': 'ny', 'Pashto': 'ps', 'Persian': 'fa', 'Polish': 'pl', 'Portuguese (Portugal, Brazil)': 'pt', 'Punjabi': 'pa', 'Romanian': 'ro', 'Russian': 'ru', 'Samoan': 'sm', 'Scots Gaelic': 'gd', 'Serbian': 'sr', 'Sesotho': 'st', 'Shona': 'sn', 'Sindhi': 'sd', 'Sinhala (Sinhalese)': 'si', 'Slovak': 'sk', 'Slovenian': 'sl', 'Somali': 'so', 'Spanish': 'es', 'Sundanese': 'su', 'Swahili': 'sw', 'Swedish': 'sv', 'Tagalog (Filipino)': 'tl', 'Tajik': 'tg', 'Tamil': 'ta', 'Telugu': 'te', 'Thai': 'th', 'Turkish': 'tr', 'Ukrainian': 'uk', 'Urdu': 'ur', 'Uzbek': 'uz', 'Vietnamese': 'vi', 'Welsh': 'cy', 'Xhosa': 'xh', 'Yiddish': 'yi', 'Yoruba': 'yo', 'Zulu': 'zu'}


%cd {repo_path}
# Applying the selected language and setting the prompt size to 2048
# if 8bit mode is selected
j = json.loads(open('settings-colab.json', 'r').read())
j["google_translate-language string"] = language_codes[chat_language]
if load_in_8bit:
  j["chat_prompt_size"] = 2048
with open('settings-colab.json', 'w') as f:
  f.write(json.dumps(j, indent=4))

params = set()
if chat:
  params.add('--chat')

if public_api:
  params.add('--public-api')

if api:
  params.add('--api')

if load_in_8bit:
  params.add('--load-in-8bit')
#if auto_devices:
#  params.add('--auto-devices')
if load_4bit_models:
  params.add('--wbits 4')

if groupsize_128:
  params.add('--groupsize 128')

active_extensions = []
if activate_sending_pictures:
  active_extensions.append('send_pictures')
if activate_character_bias:
  active_extensions.append('character_bias')
if activate_google_translate:
  active_extensions.append('google_translate')
if activate_silero_text_to_speech:
  active_extensions.append('silero_tts')
#active_extensions.append('gallery')

if len(active_extensions) > 0:
  params.add(f'--extensions {" ".join(active_extensions)}')

if not text_streaming or activate_google_translate or activate_silero_text_to_speech:
  params.add('--no-stream')
if activate_character_bias:
  params.add('--verbose')

# Starting the web UI
cmd = f"python server.py --share  --model {model_load} --settings settings-colab.json {' '.join(params)}"
print(cmd)
!$cmd